<a href="https://colab.research.google.com/github/Afnaan-Alqadree/ai-ml-projects/blob/main/Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505222 sha256=1b09e7b4ad344bcd83ef3aa057e3a085642272bcc6e29f8e09bb943c53a02b0f
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [1]:
!pip uninstall -y numpy
!pip install numpy==1.24.4
!pip install scikit-surprise

Found existing installation: numpy 1.24.4
Uninstalling numpy-1.24.4:
  Successfully uninstalled numpy-1.24.4
  Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.0 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is in

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 362, in run
^C


In [1]:
import pandas as pd

# Convert u.data to ratings.csv
ratings_columns = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=ratings_columns)
ratings.to_csv('ratings.csv', index=False)
print("✅ ratings.csv created!")

# Convert u.item to movies.csv
movies_columns = [
    "movie_id", "title", "release_date", "video_release_date", "IMDb_URL",
    "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime",
    "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery",
    "Romance", "Sci-Fi", "Thriller", "War", "Western"
]
movies = pd.read_csv('u.item', sep='|', names=movies_columns, encoding='latin-1')
movies.to_csv('movies.csv', index=False)
print("✅ movies.csv created!")


✅ ratings.csv created!
✅ movies.csv created!


In [7]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd

# Step 1: Load your ratings.csv
ratings_df = pd.read_csv("ratings.csv")
# Load movies.csv
movies_df = pd.read_csv("movies.csv")

# Step 2: Load data using surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df[['user_id', 'movie_id', 'rating']], reader)



In [8]:
from surprise.model_selection import GridSearchCV
from surprise import KNNWithMeans

# Define the parameter grid
param_grid = {
    "sim_options": {
        "name": ["cosine", "msd"],
        "user_based": [True, False],
        "min_support": [3, 5]
    }
}

# Create the GridSearchCV object
gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# Output the best score and parameters
print("✅ Best RMSE:", gs.best_score['rmse'])
print("✅ Best Params:", gs.best_params['rmse'])

# Now use the best model
algo = gs.best_estimator['rmse']


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the m

In [12]:
from surprise import SVD

# Define the parameter grid for SVD
param_grid_svd = {
    "n_epochs": [5, 10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6]
}

# Create GridSearchCV object for SVD
gs_svd = GridSearchCV(SVD, param_grid_svd, measures=["rmse", "mae"], cv=3)
gs_svd.fit(data)

# Output best score and parameters
print("✅ Best RMSE (SVD):", gs_svd.best_score['rmse'])
print("✅ Best Params (SVD):", gs_svd.best_params['rmse'])

# Use the best SVD model
algo_svd = gs_svd.best_estimator['rmse']


✅ Best RMSE (SVD): 0.9635273262787454
✅ Best Params (SVD): {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [13]:
# Compare RMSE from both models
if gs.best_score['rmse'] < gs_svd.best_score['rmse']:
    print("Using KNNWithMeans model")
    algo = gs.best_estimator['rmse']
else:
    print("Using SVD model")
    algo = algo_svd


Using KNNWithMeans model


In [14]:

trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# Fit the best algorithm
algo.fit(trainset)

# Predict on test set
predictions = algo.test(testset)

# Evaluate
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse:.4f}")

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9391
RMSE: 0.9391


In [20]:
from collections import defaultdict

# Get top N recommendations for all users
def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Keep top N for each user
    for uid, user_ratings in top_n.items():
        top_n[uid] = sorted(user_ratings, key=lambda x: x[1], reverse=True)[:n]

    return top_n

top_n = get_top_n(predictions, n=10)



# Show top 10 recommended movies for a specific user (e.g., User 1)
user_id = 1

print(f"\n🎬 Top 10 movie recommendations for User {user_id}:")
print(f"{'Movie ID':<10} {'Title':<40} {'Predicted Rating'}")
print("=" * 70)

for movie_id, score in top_n[user_id]:
    # Get movie title using movie_id
    title = movies_df[movies_df['movie_id'] == movie_id]['title'].values
    title = title[0] if len(title) > 0 else "Unknown Title"

    print(f"{movie_id:<10} {title:<40} {score:.2f}")



🎬 Top 10 movie recommendations for User 1:
Movie ID   Title                                    Predicted Rating
199        Bridge on the River Kwai, The (1957)     4.62
98         Silence of the Lambs, The (1991)         4.62
251        Shall We Dance? (1996)                   4.60
190        Henry V (1989)                           4.58
134        Citizen Kane (1941)                      4.56
170        Cinema Paradiso (1988)                   4.49
195        Terminator, The (1984)                   4.49
180        Apocalypse Now (1979)                    4.45
56         Pulp Fiction (1994)                      4.45
59         Three Colors: Red (1994)                 4.34
